
# Loading and Preprocessing the Data

In [1]:
!unzip -qq 'Dataset.zip'

In [2]:
import cv2
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from matplotlib import pyplot as plt

In [3]:
train = tf.keras.utils.image_dataset_from_directory(
    'traffic_light_data/train',
    labels='inferred' ,
    label_mode = 'int',
    color_mode="rgb",
    batch_size=1,
    image_size=(32, 32),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)


test = tf.keras.utils.image_dataset_from_directory(
    'traffic_light_data/val',
    labels='inferred' ,
    label_mode = 'int',
    color_mode="rgb",
    batch_size=1,
    image_size=(32, 32),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 3046 files belonging to 3 classes.
Found 597 files belonging to 3 classes.


In [4]:
x_train = np.empty((1,32,32,3), dtype= np.int8, order='C')
y_train = np.empty((1,1), dtype= np.int8, order='C')
for image_batch, labels_batch in train:
  x_train = np.vstack((x_train, image_batch))
  y_train = np.vstack((y_train, labels_batch))

In [5]:
x_test = np.empty((1,32,32,3), dtype=np.int8, order='C')
y_test = np.empty((1,1), dtype=np.int8, order='C')
for image_batch, labels_batch in test:
  x_test = np.vstack((x_test, image_batch))
  y_test = np.vstack((y_test, labels_batch))

In [6]:
x_train = x_train[1:]
y_train = y_train[1:]
x_test = x_test[1:]
y_test = y_test[1:]

# Extracting Color Gradient Histograms

In [7]:
x_train_hist = [cv2.calcHist([x_train[0]], [0,1,2], None, [64,64,64], [0,256,0,256,0,256]).flatten()]
x_test_hist = [cv2.calcHist([x_test[0]], [0,1,2], None, [64,64,64], [0,256,0,256,0,256]).flatten()]

In [8]:
for i in range (len(x_train)-1):
  x = cv2.calcHist([x_train[i+1]], [0,1,2], None, [64,64,64], [0,256,0,256,0,256]).flatten()
  x_train_hist = np.vstack((x_train_hist, x))
for i in range (len(x_test)-1):
  x = cv2.calcHist([x_test[i+1]], [0,1,2], None, [64,64,64], [0,256,0,256,0,256]).flatten()
  x_test_hist = np.vstack((x_test_hist, x))


# Normalizing the Data

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_hist = sc.fit_transform(x_train_hist)
x_test_hist = sc.transform(x_test_hist)

# Classification using Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(max_iter=1000)

logisticRegr.fit(x_train_hist, np.ravel(y_train))

LogisticRegression(max_iter=1000)

In [11]:
y_pred = logisticRegr.predict(x_test_hist)

# Final Result

In [12]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


accuracy = accuracy_score(np.ravel(y_test), y_pred)
print('Accuracy: ' , accuracy)


Accuracy:  0.9782244556113903


In [14]:
matrix = confusion_matrix(np.ravel(y_test), y_pred)
print(matrix,'\n')
acc = matrix.diagonal()/matrix.sum(axis=1)
print('Red accuracy = ', acc[1],'\n')
print('Yellow accuracy = ', acc[2],'\n')
print('Green accuracy = ', acc[0],'\n')

[[206   0   1]
 [  0 281   0]
 [  9   3  97]] 

Red accuracy =  1.0 

Yellow accuracy =  0.8899082568807339 

Green accuracy =  0.9951690821256038 

